In [1]:
import pandas as pd
import numpy as np
import sys
import string
from string import digits
from pandas import Panel
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import os
import re
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et 
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

C:\Users\Hasibur1\Anaconda3_\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  
Using TensorFlow backend.


In [2]:
# import pandas as pd 
import pandas as pd 
   
# Takes the file's folder 
filepath = r"C:Documents\dataset.csv"
# read the CSV file 
lines = pd.read_csv(filepath, sep='delimiter', header=None) 


C:\Users\Hasibur1\Anaconda3_\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


In [3]:
print(len(lines)) 
lines.columns = ['qa']
list(lines.columns)

4040


['qa']

In [4]:
lines[['question','answer']] = lines.qa.apply(lambda x: pd.Series(str(x).split("?")))

In [5]:
list(lines.columns)

['qa', 'question', 'answer']

In [6]:
for col in lines.columns: 
    if 'qa' in col: 
        del lines[col] 
  

In [7]:
lines.head()

,question,answer
0,What is (are) abdominal wall defect,",An opening in the abdomen through which vario..."
1,What are the treatments for abdominal wall defect,",Diagnostic Tests-Drug Therapy-Surgery and Reh..."
2,What is (are) Osteoporosis,",A Bone Disease that thins and weakens the bon..."
3,Who is at risk for Osteoporosis,",Women are at higher risk for osteoporosis tha..."
4,What are the symptoms of Osteoporosis,",Fractures-A Possible Warning Sign Osteoporosi..."


In [8]:
lines.columns = ['source', 'target']

In [9]:
lines['target'] = lines['target'].fillna('').apply(str)

In [10]:
# convert source and target text to Lowercase 
lines.source=lines.source.apply(lambda x: x.lower())
lines.target=lines.target.apply(lambda x: x.lower())

# Remove quotes from source and target text
lines.source=lines.source.apply(lambda x: re.sub("'", '', x))
lines.target=lines.target.apply(lambda x: re.sub("'", '', x))

# create a set of all special characters
special_characters= set(string.punctuation)

# Remove all the special characters
lines.source = lines.source.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))
lines.target = lines.target.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))

# Remove digits from source and target sentences
num_digits= str.maketrans('','', digits)
lines.source=lines.source.apply(lambda x: x.translate(num_digits))
lines.target= lines.target.apply(lambda x: x.translate(num_digits))

# Remove extra spaces
lines.source=lines.source.apply(lambda x: x.strip())
lines.target=lines.target.apply(lambda x: x.strip())
lines.source=lines.source.apply(lambda x: re.sub(" +", " ", x))
lines.target=lines.target.apply(lambda x: re.sub(" +", " ", x))

In [11]:
# Add start and end tokens to target sequences
lines.target = lines.target.apply(lambda x : 'START_ '+ x + ' _END')
lines.sample(6)

,source,target
1326,what research or clinical trials is being done...,START_ multiple studies are looking at ways to...
742,how many people are affected by phosphoribosyl...,START_ prs superactivity is believed to be a r...
1042,is aarskogscott syndrome inherited,START_ aarskogscott syndrome is inherited in a...
3148,what is are snyderrobinson syndrome,START_ a condition characterized by intellectu...
715,how many people are affected by peutzjeghers s...,START_ the prevalence of this condition is unc...
3908,is beta hydroxysteroid dehydrogenase deficienc...,START_ this condition is inherited in an autos...


In [12]:
# Find all the source and target words and sort them
# Vocabulary of Source language
all_source_words=set()
for source in lines.source:
    for word in source.split():
        if word not in all_source_words:
            all_source_words.add(word)
# Vocabulary of Target 
all_target_words=set()
for target in lines.target:
    for word in target.split():
        if word not in all_target_words:
            all_target_words.add(word)
# sort all unique source and target words
source_words= sorted(list(all_source_words))
target_words=sorted(list(all_target_words))

len(source_words), len(target_words)

(1193, 3880)

In [13]:
#Find maximum sentence length in  the source and target data
source_length_list=[]
for l in lines.source:
    source_length_list.append(len(l.split(' ')))
max_source_length= max(source_length_list)
print(" Max length of the source sentence",max_source_length)
target_length_list=[]
for l in lines.target:
    target_length_list.append(len(l.split(' ')))
max_target_length= max(target_length_list)
print(" Max length of the target sentence",max_target_length)

 Max length of the source sentence 18
 Max length of the target sentence 31


In [14]:
# creating a word to index(word2idx) for source and target
source_word2idx= dict([(word, i+1) for i,word in enumerate(source_words)])
target_word2idx=dict([(word, i+1) for i, word in enumerate(target_words)])

In [15]:
#creating a dictionary for index to word for source and target vocabulary
source_idx2word= dict([(i, word) for word, i in  source_word2idx.items()])
print(source_idx2word)
target_idx2word =dict([(i, word) for word, i in target_word2idx.items()])

{1: 'a', 2: 'aarskogscott', 3: 'ab', 4: 'abdominal', 5: 'abetalipoproteinemia', 6: 'abnormal', 7: 'abuse', 8: 'acatalasemia', 9: 'achondrogenesis', 10: 'achondroplasia', 11: 'achromatopsia', 12: 'acid', 13: 'acidemia', 14: 'acidosis', 15: 'aciduria', 16: 'acral', 17: 'acrofacial', 18: 'acromicric', 19: 'actinaccumulation', 20: 'activated', 21: 'acute', 22: 'acylcoa', 23: 'adamsoliver', 24: 'adcyrelated', 25: 'adenine', 26: 'adenosine', 27: 'adenylosuccinate', 28: 'adermatoglyphia', 29: 'adhesion', 30: 'adiposis', 31: 'adolescent', 32: 'adrenal', 33: 'adrenoleukodystrophy', 34: 'adult', 35: 'adultonset', 36: 'adults', 37: 'affected', 38: 'african', 39: 'agammaglobulinemia', 40: 'agerelated', 41: 'aggregate', 42: 'aicardi', 43: 'aicardigoutieres', 44: 'aidsrelated', 45: 'alagille', 46: 'albinism', 47: 'alexander', 48: 'algcongenital', 49: 'alkaptonuria', 50: 'allanherndondudley', 51: 'allergic', 52: 'alopecia', 53: 'alopeciaand', 54: 'alpershuttenlocher', 55: 'alpha', 56: 'alphabeta', 57

In [16]:
#Shuffle the data
lines = shuffle(lines)

In [20]:
# Train - Test Split
X, y = lines.source, lines.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.01)
X_train.shape, X_test.shape

((3999,), (41,))

In [21]:
# https://stackoverflow.com/questions/54880279/how-to-handle-invalid-argument-error-in-keras
# Input tokens for encoder
num_encoder_tokens=len(source_words) +1
# Input tokens for decoder zero padded
num_decoder_tokens=len(target_words) +1

In [22]:
def generate_batch(X = X_train, y = y_train, batch_size = 180):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_source_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_target_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_target_length, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                        encoder_input_data[i, t] = source_word2idx[word] 
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_word2idx[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        #print(word)
                        decoder_target_data[i, t - 1, target_word2idx[word]] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [23]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100
latent_dim=256

In [24]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [25]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [26]:
# Define the model that takes encoder and decoder input 
# to output decoder_outputs
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [27]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [28]:
train_samples = len(X_train) # Total Training samples
val_samples = len(X_test)    # Total validation or test samples
batch_size = 128
epochs = 100

In [29]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = train_samples//batch_size)

C:\Users\Hasibur1\Anaconda3_\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
31/31 [==============================] - 44s 1s/step - loss: 2.1269 - acc: 0.1087 - val_loss: 0.6163 - val_acc: 0.1203
Epoch 2/100
31/31 [==============================] - 41s 1s/step - loss: 1.8377 - acc: 0.1307 - val_loss: 0.5869 - val_acc: 0.1693
Epoch 3/100
31/31 [==============================] - 40s 1s/step - loss: 1.7288 - acc: 0.1687 - val_loss: 0.5528 - val_acc: 0.2673
Epoch 4/100
31/31 [==============================] - 39s 1s/step - loss: 1.5837 - acc: 0.2521 - val_loss: 0.5176 - val_acc: 0.3385
Epoch 5/100
31/31 [==============================] - 38s 1s/step - loss: 1.4481 - acc: 0.3183 - val_loss: 0.4882 - val_acc: 0.3831
Epoch 6/100
31/31 [==============================] - 39s 1s/step - loss: 1.3321 - acc: 0.3712 - val_loss: 0.4605 - val_acc: 0.4098
Epoch 7/100
31/31 [==============================] - 39s 1s/step - loss: 1.2406 - acc: 0.4120 - val_loss: 0.4438 - val_acc: 0.4454
Epoch 8/100
31/31 [==============================] - 39s 1s/step - loss: 1.1640 - a

31/31 [==============================] - 40s 1s/step - loss: 0.1399 - acc: 0.9244 - val_loss: 0.4144 - val_acc: 0.5323
Epoch 64/100
31/31 [==============================] - 39s 1s/step - loss: 0.1344 - acc: 0.9280 - val_loss: 0.4145 - val_acc: 0.5323
Epoch 65/100
31/31 [==============================] - 40s 1s/step - loss: 0.1306 - acc: 0.9333 - val_loss: 0.4170 - val_acc: 0.5167
Epoch 66/100
31/31 [==============================] - 40s 1s/step - loss: 0.1220 - acc: 0.9371 - val_loss: 0.4219 - val_acc: 0.5367
Epoch 67/100
31/31 [==============================] - 39s 1s/step - loss: 0.1169 - acc: 0.9401 - val_loss: 0.4227 - val_acc: 0.5323
Epoch 68/100
31/31 [==============================] - 40s 1s/step - loss: 0.1110 - acc: 0.9442 - val_loss: 0.4268 - val_acc: 0.5256
Epoch 69/100
31/31 [==============================] - 41s 1s/step - loss: 0.1052 - acc: 0.9478 - val_loss: 0.4328 - val_acc: 0.5189
Epoch 70/100
31/31 [==============================] - 40s 1s/step - loss: 0.0996 - acc: 0

In [30]:
model.save_weights('nmt_weights_0epochs.h5')

In [31]:
model.load_weights('nmt_weights_0epochs.h5')

In [32]:
# Encode the input sequence to get the "Context vectors"
encoder_model = Model(encoder_inputs, encoder_states)
# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]
# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_input)
decoder_states2 = [state_h2, state_c2]
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)
# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_state_input,
    [decoder_outputs2] + decoder_states2)

In [33]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of 
    #target sequence with the start character.
    target_seq[0, 0] = target_word2idx['START_']
# Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
# Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word =target_idx2word[sampled_token_index]
        decoded_sentence += ' '+ sampled_word
# Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '_END' or
           len(decoded_sentence) > 120):
            stop_condition = True
# Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
# Update states
        states_value = [h, c]
    return decoded_sentence

In [34]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [35]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_train[k:k+1].values[0])
print('Actual Target Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])

Input Source sentence: what are the genetic changes related to head and neck squamous cell carcinoma
Actual Target Translation:  hnscc is caused by a variety of factors that can alter the dna in cells 
Predicted Target Translation:  hnscc is caused by a variety of factors that can alter the dna in cells 


In [36]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=10
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_test[k:k+1].values[0])
print('Actual Target Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])

Input Source sentence: what is are hereditary sensory neuropathy type ia
Actual Target Translation:  hereditary sensory neuropathy type ia is a condition characterized by nerve abnormalities in the legs and feet 
Predicted Target Translation:  stage istage iistage iii 
